In [1]:
# importing packages
from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# font size
sns.set_style("white", rc={"font_scale": 1.5})

# loforest and locart functions
from CP2LFI.loforest import ConformalLoforest
from CP2LFI.scores import LambdaScore

from clover import Scores
from clover import LocartSplit

from copy import deepcopy
from tqdm import tqdm

from scipy import stats
from scipy.optimize import minimize_scalar

import time
import itertools

\section{Comparing our methods for a BFF example with normal distribution}
Taking $(\mu, \sigma^2) \in \Theta = [-5,5] \times (0, 1]$ we consider a normal-inverse gamma prior over $(\mu, \sigma^2)$ given by:
\begin{align*}
\mu|\sigma^2 \sim N(0, 0.25 \sigma^2) , \\
\sigma^2 \sim IG(2 ,1) ,
\end{align*}
with the likelihood of $X$ given by:
\begin{align*}
X|\mu, \sigma^2 \sim N(\mu, \sigma^2).
\end{align*}
In this case, we obtain the $1 - \alpha$ credibility region of $\theta$ by obtaining $C_{\theta}$ such that:
\begin{align*}
\mathbb{P}(\{(\mu, \sigma^2): f(\mu, \sigma^2|x) \geq C_{\left( \mu, \sigma^2 \right)}\}) = 1 - \alpha .
\end{align*}

Now we define the main functions to simulate all samples and compute naive quantiles:

In [2]:
def sim_X(n, mu, sigma_2):
    X = np.random.normal(mu, np.sqrt(sigma_2), n)
    return X


def sim_lambda(B, N, mu, sigma_2, l=4, a=2, beta=1):
    lambdas = np.zeros(B)
    for i in range(0, B):
        X = sim_X(N, mu, sigma_2)
        lambdas[i] = compute_pdf_posterior(
            mu,
            sigma_2,
            X,
            l=l,
            a=a,
            beta=beta,
        )
    return lambdas


def sample_posterior(B, N, seed=45, l=4, a=2, beta=1):
    np.random.seed(seed)
    thetas = np.c_[np.random.uniform(-5, 5, B), np.random.uniform(0, 1, B)]
    lambdas = np.zeros(n)
    i = 0
    for mu, sigma_2 in thetas:
        X = sim_X(N, mu, sigma_2)
        lambdas[i] = compute_pdf_posterior(
            mu,
            sigma_2,
            X,
            l=l,
            a=a,
            beta=beta,
        )
        i += 1
    return thetas, lambdas


def compute_pdf_posterior(mu, sigma_2, x, l=4, a=2, beta=1):
    n = x.shape[0]
    sum_squares = n * np.var(x)
    x_bar = np.mean(x)

    # posterior parameters
    mu_value = (1 / (l + n)) * (np.sum(x))
    l_value = l + n
    alpha_value = a + (n / 2)
    beta_value = (
        beta + ((1 / 2) * (sum_squares)) + ((n * l) / (l + n) * ((x_bar**2) / 2))
    )

    # pdf for mu
    f_mu = stats.norm.pdf(mu, loc=mu_value, scale=np.sqrt(sigma_2 / l_value))

    f_sigma_2 = stats.invgamma.pdf(sigma_2, a=alpha_value, scale=beta_value)

    return -(f_mu * f_sigma_2)


# naive method
def naive(
    alpha,
    B=1000,
    N=100,
    lower=-5,
    upper=5,
    seed=250,
    naive_n=100,
    l=4,
    a=2,
    beta=1,
):
    n_grid = int(B / naive_n)
    mu = np.linspace(-5, 5, n_grid)
    sigma_2 = np.linspace(0.0001, 1, n_grid)
    quantiles = {}
    for mu, sigma_2 in itertools.product(mu, sigma_2):
        lambdas = sim_lambda(
            B=int(np.sqrt(naive_n)),
            N=N,
            mu=mu,
            sigma_2=sigma_2,
            l=l,
            a=a,
            beta=beta,
        )
        quantiles[(mu, sigma_2)] = np.quantile(lambdas, q=1 - alpha)
    return quantiles


# naive predict function
def predict_naive_quantile(theta_grid, quantiles_dict):
    thetas_values = np.array(list(quantiles_dict.keys()))
    quantiles_list = []
    for x in theta_grid:
        distances = np.linalg.norm(thetas_values - x, axis=1)
        idx = thetas_values[np.argmin(distances)]
        quantiles_list.append(quantiles_dict[tuple(idx)])
    return quantiles_list

Lets now compare our methods to all the others for several N:

In [3]:
def obtain_quantiles(
    thetas,
    N,
    B=1000,
    alpha=0.05,
    naive_seed=45,
    min_samples_leaf=100,
    naive_n=500,
    sample_seed=25,
    l=4,
    a=2,
    beta=1,
):
    # fitting and predicting naive
    naive_quantiles = naive(
        alpha=alpha,
        B=B,
        N=N,
        naive_n=naive_n,
        sigma=sigma,
        seed=naive_seed,
        l=l,
        a=a,
        beta=beta,
    )

    # simulating to fit models
    model_thetas, model_lambdas = sample_posterior(n=B, N=N, seed=sample_seed)

    locart_object = LocartSplit(
        LambdaScore, None, alpha=alpha, is_fitted=True, split_calib=False
    )
    locart_quantiles = locart_object.calib(
        model_thetas, model_lambdas, min_samples_leaf=min_samples_leaf
    )

    # loforest quantiles
    loforest_object = ConformalLoforest(
        LambdaScore, None, alpha=alpha, is_fitted=True, split_calib=False
    )
    loforest_object.calibrate(
        model_thetas, model_lambdas, min_samples_leaf=min_samples_leaf
    )

    # boosting quantiles
    model = HistGradientBoostingRegressor(
        loss="quantile",
        max_iter=100,
        max_depth=3,
        quantile=1 - alpha,
        random_state=105,
        n_iter_no_change=15,
        early_stopping=True,
    )
    model.fit(model_thetas, model_lambdas)

    # naive quantiles
    naive_list = predict_naive_quantile(thetas, naive_quantiles)

    # locart quantiles
    idxs = locart_object.cart.apply(thetas.reshape(-1, 1))
    list_locart_quantiles = [locart_quantiles[idx] for idx in idxs]

    # loforest
    loforest_cutoffs = loforest_object.compute_cutoffs(thetas.reshape(-1, 1))

    # boosting
    boosting_quantiles = model.predict(thetas.reshape(-1, 1))

    # dictionary of quantiles
    quantile_dict = {
        "naive": naive_list,
        "locart": list_locart_quantiles,
        "loforest": loforest_cutoffs,
        "boosting": boosting_quantiles,
    }

    return quantile_dict


# evaluate coverage for several N's and B = 1000
def evaluate_coverage_N(
    thetas,
    N=np.array([10, 100, 1000]),
    B=1000,
    alpha=0.05,
    n=1000,
    seed=45,
    min_samples_leaf=100,
    naive_n=100,
    l=4,
    a=2,
    beta=1,
):
    coverage_data = np.zeros((thetas.shape[0] * N.shape[0], 4))
    N_list = []
    N_list_cover = []
    methods_list = []
    np.random.seed(seed)
    seeds = np.random.choice(
        np.arange(0, 10**4, 1),
        N.shape[0],
        replace=False,
    )
    sample_seeds = np.random.choice(
        np.arange(0, 10**4, 1),
        N.shape[0],
        replace=False,
    )
    k = 0
    j = 0
    for N_fixed in tqdm(N, desc="Computing coverage for each N"):
        # computing all quantiles for fixed N
        quantiles_dict = obtain_quantiles(
            thetas,
            N=N_fixed,
            B=B,
            alpha=alpha,
            naive_seed=seeds[k],
            min_samples_leaf=min_samples_leaf,
            naive_n=naive_n,
            sample_seed=sample_seeds[k],
            l=l,
            a=a,
            beta=beta,
        )
        err_data = np.zeros((thetas.shape[0], 4))
        l = 0
        for theta in tqdm(thetas, desc="Computing coverage for each method"):
            # generating several lambdas
            lambda_stat = sim_lambda(B=n, N=N_fixed, theta=theta, sigma=sigma)

            # comparing coverage of methods
            locart_cover = np.mean(lambda_stat <= quantiles_dict["locart"][l])
            loforest_cover = np.mean(lambda_stat <= quantiles_dict["loforest"][l])
            boosting_cover = np.mean(lambda_stat <= quantiles_dict["boosting"][l])
            naive_cover = np.mean(lambda_stat <= quantiles_dict["naive"][l])

            # appending the errors
            err_locart = np.abs(locart_cover - (1 - alpha))
            err_loforest = np.abs(loforest_cover - (1 - alpha))
            err_boosting = np.abs(boosting_cover - (1 - alpha))
            err_naive = np.abs(naive_cover - (1 - alpha))

            # saving in numpy array
            err_data[l, :] = np.array(
                [err_locart, err_loforest, err_boosting, err_naive]
            )
            N_list_cover.append(N_fixed)

            j += 1
            l += 1
        methods_list.extend(["LOCART", "LOFOREST", "boosting", "naive"])
        if k == 0:
            mae_vector = np.mean(err_data, axis=0)
            std_vector = np.std(err_data, axis=0) / (np.sqrt(thetas.shape[0]))
        else:
            mean = np.mean(err_data, axis=0)
            std = np.std(err_data, axis=0) / (np.sqrt(thetas.shape[0]))
            mae_vector, std_vector = np.concatenate((mae_vector, mean)), np.concatenate(
                (std_vector, std)
            )
        k += 1
        N_list.extend([N_fixed] * 4)

    # obtaining MAE and standard error for each method
    stats_data = pd.DataFrame(
        {
            "methods": methods_list,
            "N": N_list,
            "MAE": mae_vector,
            "se": std_vector,
        }
    )

    coverage_data = pd.DataFrame(
        {
            "thetas": np.tile(thetas, N.shape[0]),
            "N": N_list_cover,
            "LOCART": coverage_data[:, 0],
            "LOFOREST": coverage_data[:, 1],
            "boosting": coverage_data[:, 2],
            "naive": coverage_data[:, 3],
        }
    )
    return [stats_data, coverage_data]

Testing for $B = 5000$:


In [4]:
n_out = 50
mu_s = np.linspace(-5, 5, n_out)
sigmas = np.linspace(0.0001, 1, n_out)
thetas_grid = np.c_[list(itertools.product(mu_s, sigmas))]
# printing grid size
thetas_grid.shape[0]

2500